### Resizing images to have a consistent ratio

This is needed for the dash app, otherwise some pictures (that are much longer than they are wide) take up a very large amount of space in order to fill out the card.

By making everything square, the sizing of each box should be the same too.

Adapated from: https://stackoverflow.com/questions/44370469/python-image-resizing-keep-proportion-add-white-background

But going to make background transparent too.

Manually made a backup of the assets folder with all current images.

In [1]:
from typing import Tuple

from PIL import Image
import sqlite3
import pandas as pd

In [2]:
# read in database
DATABASE_LOC = r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\PyForFun\House_Plant_Recommender\Database\house_plants.db"

conn = sqlite3.connect(DATABASE_LOC)
c = conn.cursor()
image_df = pd.read_sql_query("SELECT * FROM 'plant_images'", conn)
c.close()

In [3]:
image_df

,Plant_Name,File_Path,Website
0,Aechmea,assets/Aechmea.jpg,commons.wikimedia.org
1,Ardisia crenata,assets/Ardisia_crenata.jpg,commons.wikimedia.org
2,Euphorbia milii,assets/Euphorbia_milii.jpg,commons.wikimedia.org
3,Ficus elastica,assets/Ficus_elastica.jpg,commons.wikimedia.org
4,Woodsia obtusa,assets/Woodsia_obtusa.jpg,commons.wikimedia.org
...,...,...,...
142,Begonia,assets/Begonia.jpg,commons.wikimedia.org
143,Ficus carica,assets/Ficus_carica.jpg,commons.wikimedia.org
144,Ficus pumila,assets/Ficus_pumila.jpg,commons.wikimedia.org
145,Ficus religiosa,assets/Ficus_religiosa.jpg,commons.wikimedia.org


In [4]:
def reformat_image(image_in_path:str, image_out_path:str) -> bool:
    """   
    Reformat each image so that size and width are the same. 
    This is done by calculating smaller side and adding transparent blocks 
    to each end of the smaller side (so image still in the center). 
    """
    image = Image.open(image_in_path, "r")
    image_size = image.size
    width = image_size[0]
    height = image_size[1]

    
    if(width != height):
        bigside = width if width > height else height

        background = Image.new("RGBA", (bigside, bigside), (255, 255, 255, 0))
        offset = (int(round(((bigside - width) / 2), 0)), int(round(((bigside - height) / 2),0)))

        background.paste(image, offset)
        background.save(image_out_path)
        return True
    else:
        return False 

In [5]:
# Have to convert these to .png files as .jpg does not support transparent pictures 
changed_images, unchanged_images = [], []
for image_path_raw, plant_name in zip(list(image_df["File_Path"]), list(image_df["Plant_Name"])):
    image_path_corrected = "../" + image_path_raw
    image_out_path = image_path_corrected.split(".jpg")[0] + "_resized.png"

    result = reformat_image(
        image_in_path=image_path_corrected,
        image_out_path=image_out_path,
    )

    if result: # if new file made.
        image_path_dash =  image_path_raw.split(".jpg")[0] + "_resized.png" # works for Dash app's location. 
        changed_images.append( (image_path_dash, plant_name) )
    else:
        unchanged_images.append( (image_path_raw, plant_name) )

print(f"Number of images reformatted: {len(changed_images)}") 

Number of images reformatted: 145


In [6]:
changed_images

[('assets/Aechmea_resized.png', 'Aechmea'),
 ('assets/Ardisia_crenata_resized.png', 'Ardisia crenata'),
 ('assets/Euphorbia_milii_resized.png', 'Euphorbia milii'),
 ('assets/Ficus_elastica_resized.png', 'Ficus elastica'),
 ('assets/Woodsia_obtusa_resized.png', 'Woodsia obtusa'),
 ('assets/Kalanchoe_resized.png', 'Kalanchoe'),
 ('assets/Kalanchoe_thyrsiflora_resized.png', 'Kalanchoe thyrsiflora'),
 ('assets/Tradescantia_spathacea_resized.png', 'Tradescantia spathacea'),
 ('assets/Capsicum_annuum_resized.png', 'Capsicum annuum'),
 ('assets/Begonia_masoniana_resized.png', 'Begonia masoniana'),
 ('assets/Aloe_resized.png', 'Aloe'),
 ('assets/Oncidium_resized.png', 'Oncidium'),
 ('assets/Zantedeschia_resized.png', 'Zantedeschia'),
 ('assets/Epipremnum_aureum_resized.png', 'Epipremnum aureum'),
 ('assets/Hedera_helix_resized.png', 'Hedera helix'),
 ('assets/Sedum_makinoi_resized.png', 'Sedum makinoi'),
 ('assets/Asplenium_nidus_resized.png', 'Asplenium nidus'),
 ('assets/Elettaria_cardamomum

In [7]:
unchanged_images

[('assets/Monstera_deliciosa.jpg', 'Monstera deliciosa'),
 ('assets/Aglaonema.jpg', 'Aglaonema')]

### Now update there paths in the database. 

In [8]:

conn = sqlite3.connect(DATABASE_LOC)
c = conn.cursor()
c.executemany("UPDATE plant_images SET File_Path=? WHERE Plant_Name=?", changed_images)


In [9]:
# check if correctly updated before commiting... 
image_df_2 = pd.read_sql_query("SELECT * FROM 'plant_images'", conn)
image_df_2

,Plant_Name,File_Path,Website
0,Aechmea,assets/Aechmea_resized.png,commons.wikimedia.org
1,Ardisia crenata,assets/Ardisia_crenata_resized.png,commons.wikimedia.org
2,Euphorbia milii,assets/Euphorbia_milii_resized.png,commons.wikimedia.org
3,Ficus elastica,assets/Ficus_elastica_resized.png,commons.wikimedia.org
4,Woodsia obtusa,assets/Woodsia_obtusa_resized.png,commons.wikimedia.org
...,...,...,...
142,Begonia,assets/Begonia_resized.png,commons.wikimedia.org
143,Ficus carica,assets/Ficus_carica_resized.png,commons.wikimedia.org
144,Ficus pumila,assets/Ficus_pumila_resized.png,commons.wikimedia.org
145,Ficus religiosa,assets/Ficus_religiosa_resized.png,commons.wikimedia.org


In [10]:
# looks good, done. 
conn.commit()
c.close()